# Charlando con mi CLI vía Cisco pyATS 🚀
---

🔁 Antes de comenzar, es necesario crear un entorno virtual con los siguientes comandos:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

### 1️⃣ Creando mi archivo de topología
Ahora, toca crear el archivo `topology.yaml` donde especificaremos cuáles son nuestros dispositivos, y cómo podemos conectarnos a ellos.</br>

### 2️⃣ Leyendo mis configs
Excelente. Ahora es momento de ponerlos el sombrero de programadores y escribir algo de código.</br>
Vamos a conectarnos a nuestro dispositivo mencionado en el archivo `testbed.yaml`.</br>
pyATS usará la biblioteca `unicon` y la información en el archivo yaml para intentar crear una conexión SSH con el dispositivo `sandbox1`.

In [1]:
import json
from pyats.topology import loader

In [13]:
testbed = loader.load('pyATS/testbed.yaml')
device = testbed.devices['sandbox1']

device.connect()
print("✅ Connected successfully to", device.name)


2025-11-14 13:11:18,832: %UNICON-INFO: +++ sandbox1 logfile sandbox1-cli-1763122278.log +++

2025-11-14 13:11:18,833: %UNICON-INFO: +++ Unicon plugin iosxr (unicon.plugins.iosxr) +++

Hello there! Hoping you are having a great day
... Welcome to 'ios',
your favorite CISCO.IOSXR.IOSXR Sandbox



2025-11-14 13:11:20,257: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.150 -p 22, id: 4691545744 +++

2025-11-14 13:11:20,259: %UNICON-INFO: connection to sandbox1
(admin@131.226.217.150) Password: 


RP/0/RP0/CPU0:ios#

2025-11-14 13:11:21,202: %UNICON-INFO: Storing credentials from default as current_credentials

2025-11-14 13:11:21,203: %UNICON-WARNING: Invalid hostname detected: found ios, expected sandbox1

2025-11-14 13:11:21,204: %UNICON-INFO: +++ initializing handle +++

2025-11-14 13:11:21,401: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'terminal length 0' +++
terminal length 0
Fri Nov 14 12:58:45.521 UTC
RP/0/RP0/CPU0:ios#

2025-11-14 13:11:21,948: 

🔌 Excelso. Lo que queremos ahora es saber **cuáles son las interfaces de este dispositivo**.</br>
**Usemos [los parsers Genie que vienen incluídos 🧞](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers)** para conseguir esta información en un JSON.

In [14]:
genie_parsed_interfaces = device.parse("show ip interface brief")
print(f"\n\n\n\n\n{json.dumps(genie_parsed_interfaces,indent=4)}")


2025-11-14 13:11:28,621: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show ip interface brief' +++
show ip interface brief
Fri Nov 14 12:58:52.740 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback1                      unassigned      Up              Up       default 
Loopback2                      unassigned      Up              Up       default 
Loopback3                      unassigned      Up              Up       default 
Loopback5                      unassigned      Up              Up       default 
Loopback6                      unassigned      Up              Up       default 
Loopback7                      unassigned      Up              Up       default 
Loopback10                     10.10.10.10     Up              Up       default 
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
Loopback999         

### 3️⃣ Hacer y deshacer una interfaz
🤖 Ahora usemos [las clases de pyATS](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/models) para crear una interfaz Loopback. pyATS se encargará de todos los pasos intermedios para crear la interfaz. Nosotros solamente tenemos qué proveer la información mínima necesaria.

In [15]:
from genie.conf.base import Interface

new_iosxr_interface = Interface(name="Loopback505", device=device)
new_iosxr_interface.description = "TechTilesDemo"
new_iosxr_interface.build_config()


2025-11-14 13:11:35,014: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Fri Nov 14 12:58:59.323 UTC
RP/0/RP0/CPU0:ios(config)#interface Loopback505
RP/0/RP0/CPU0:ios(config-if)# description TechTilesDemo
RP/0/RP0/CPU0:ios(config-if)# exit
RP/0/RP0/CPU0:ios(config)#commit
Fri Nov 14 12:59:00.305 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#


🔥 Finalmente, basta solo cambiar una función para deshacer esta configuración.

In [16]:
iface = device.interfaces.get("Loopback505")
rollback_config = iface.build_unconfig()
print(rollback_config)


2025-11-14 13:11:41,801: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Fri Nov 14 12:59:06.112 UTC
RP/0/RP0/CPU0:ios(config)#no interface Loopback505
RP/0/RP0/CPU0:ios(config)#commit
Fri Nov 14 12:59:06.940 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#
None
